In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
%matplotlib inline
df = pd.read_pickle('../data/data.pkl.gz')
df1= pd.read_pickle('../data/target.pkl.gz')
# ProfileReport(df)

In [2]:
df.drop_duplicates(inplace=True)
df.head()

,SESSION_ID,GA_NUM,VISITDATE,AUTHENTICATIONTYPE,AUTOFORMCOMPLETES,BLADECLICKS,BROWSER,CALLCLICKS,CTACLICKS,CHATINITIATIONS,...,OFFERPRODUCT,PAGEVIEWS,TRAFFICTYPE,TRAFFICMEDIUM,TRAFFICUSER,TRAFFICSOURCE,TOTALFORMCOMPLETES,USERTYPE,VISITORCATEGORY,WEBINARS
0,64590,2935,2024-10-01,None,0,NaN,Edge,0,0,0,...,None,1,Organic,None,external,(direct),0,None,C,NaN
1,21272,952,2024-10-01,None,0,NaN,Chrome,0,0,0,...,None,0,Paid,paid_search,external,google,0,None,P,NaN
2,99243,4317,2024-10-01,None,0,NaN,Chrome,0,0,0,...,None,0,Paid,paid_search,external,google,0,None,P,NaN
3,54042,2420,2024-10-01,None,0,NaN,Chrome,0,0,0,...,None,0,Paid,paid_search,external,google,0,None,P,NaN
4,5806,298,2024-10-01,None,0,NaN,Chrome,0,0,0,...,None,0,Paid,paid_search,internal,google,0,returning,C,NaN


In [3]:
df1.drop_duplicates(inplace=True)
df1.head()

,GA_NUM,VISITDATE,CONVERSION
0,1,2024-12-10,1
1,2,2025-01-08,0
2,3,2025-01-23,0
3,4,2024-11-05,0
4,5,2024-12-30,0


In [4]:
# removing data out of scope

In [5]:
# df = pd.read_pickle('../data/data.pkl.gz')
df = df[df.MARKETINGCHANNEL== 'Paid Search']

In [6]:
df = df.copy()
df["TIMESTAMPDATE"] = pd.to_datetime(df["EVENTTIMESTAMP"], unit="us", utc=True)

In [7]:
# + df.TOTALFORMCOMPLETES doesn't trigger the pipeline
df['HitVisit'] = df.AUTOFORMCOMPLETES + df.FORMCOMPLETES

In [8]:
df_out = df.merge(
    df1[['GA_NUM','VISITDATE','CONVERSION']],
    on='GA_NUM',
    how='left',
    suffixes=('', '_target')
)
# avoid having more records after visitdate plus conversion, the problem is there multiples record per day
df_out['valid_record'] = df_out['VISITDATE']<=df_out['VISITDATE_target']

In [9]:
df_out = df_out[df_out.valid_record]
df_out =df_out.copy()
df_out = df_out.sort_values(["GA_NUM", "VISITDATE", "TIMESTAMPDATE"])
df_out["rank_in_day"] = df_out.groupby(["GA_NUM", "VISITDATE"]).cumcount() + 1

In [10]:

df_out = df_out.copy()
df_out.reset_index(drop=True,inplace=True)
df_out = df_out.sort_values(["GA_NUM", "TIMESTAMPDATE"])

hit_seen = df_out.groupby("GA_NUM")["HitVisit"].cumsum()

df_out = df_out[(hit_seen == 0) | ((df_out["HitVisit"] == 1) & (hit_seen == 1))].copy()


In [11]:
df3 = df_out
df3 = df3.copy()
df3.drop_duplicates(inplace=True, )
df3.reset_index(inplace=True, drop=True)


In [12]:
df3 = df3.sort_values(["GA_NUM", "TIMESTAMPDATE"])

# índice del primer hit por GA_NUM
first_hit_idx = (
    df3.loc[df3["HitVisit"].eq(1)]
       .groupby("GA_NUM")["TIMESTAMPDATE"]
       .idxmin()
)

df3["is_first_hit"] = 0
df3.loc[first_hit_idx, "is_first_hit"] = 1


In [13]:
df3.is_first_hit.value_counts()

is_first_hit
0    77383
1     4917
Name: count, dtype: int64

In [14]:
import numpy as np
import pandas as pd

ID = "GA_NUM"
T  = "TIMESTAMPDATE"
Y  = "is_first_hit"        # tu columna de completado 0/1
H  = 2              # horizonte en sesiones: prueba 1 y 2

df3 = df3.sort_values([ID, T]).copy()

# índice de sesión (1..n) por usuario
df3["session_idx"] = df3.groupby(ID).cumcount() + 1

# sesión de conversión por usuario (primera vez que EVENT==1)
conv_idx = (
    df3.loc[df3[Y] == 1]
      .groupby(ID)["session_idx"]
      .min()
      .rename("conv_session_idx")
)

df3 = df3.join(conv_idx, on=ID)


In [15]:
df3

,SESSION_ID,GA_NUM,VISITDATE,AUTHENTICATIONTYPE,AUTOFORMCOMPLETES,BLADECLICKS,BROWSER,CALLCLICKS,CTACLICKS,CHATINITIATIONS,...,WEBINARS,TIMESTAMPDATE,HitVisit,VISITDATE_target,CONVERSION,valid_record,rank_in_day,is_first_hit,session_idx,conv_session_idx
0,34,1,2024-12-10,None,0,NaN,Chrome,0,0,0,...,NaN,2024-12-10 08:01:38.082699+00:00,0,2024-12-10,1,True,1,0,1,12
1,34,1,2024-12-10,None,0,NaN,Chrome,0,0,0,...,NaN,2024-12-10 08:01:38.082699+00:00,0,2024-12-10,1,True,2,0,2,12
2,35,1,2024-12-10,None,0,NaN,Chrome,0,0,0,...,NaN,2024-12-10 17:23:32.938569+00:00,0,2024-12-10,1,True,3,0,3,12
3,35,1,2024-12-10,None,0,NaN,Chrome,0,0,0,...,NaN,2024-12-10 17:23:32.938569+00:00,0,2024-12-10,1,True,4,0,4,12
4,36,1,2024-12-10,None,0,NaN,Chrome,0,0,0,...,NaN,2024-12-10 19:24:33.528457+00:00,0,2024-12-10,1,True,5,0,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82295,112788,4917,2024-12-27,None,0,NaN,Edge,0,0,0,...,NaN,2024-12-27 13:46:53.242936+00:00,0,2024-12-27,0,True,4,0,4,8
82296,112788,4917,2024-12-27,None,0,NaN,Edge,0,0,0,...,NaN,2024-12-27 13:46:57.785527+00:00,0,2024-12-27,0,True,5,0,5,8
82297,112788,4917,2024-12-27,None,0,NaN,Edge,0,0,0,...,NaN,2024-12-27 13:46:58.020839+00:00,0,2024-12-27,0,True,6,0,6,8
82298,112788,4917,2024-12-27,None,0,NaN,Edge,0,0,0,...,NaN,2024-12-27 13:46:58.022683+00:00,0,2024-12-27,0,True,7,0,7,8


# having this information we could predict, without any previous information 

In [16]:
df_unique = df3[df3.session_idx== df3.conv_session_idx]
df_unique = df_unique.copy()
df_unique.reset_index(inplace=True, drop=True)

In [17]:
# constant values
Remove = ['AUTOFORMCOMPLETES', 'BLADECLICKS', 'CONTACTME', 'DEMOREGISTRATIONS', 'FORMCOMPLETES', 'FORMTYPE', 'MARKETINGCHANNEL', 'NEWSLETTERSUBSCRIBES', 'TRAFFICTYPE', 'TOTALFORMCOMPLETES', 'WEBINARS', 'CALLCLICKS','CTACLICKS','CHATINITIATIONS','CHATVIEWS','FORMERROR','PAGEVIEWS','HitVisit','valid_record','is_first_hit', 'GA_NUM','SESSION_ID','GA_NUM','EVENTTIMESTAMP','TIMESTAMPDATE','AUTHENTICATIONTYPE', 'VISITDATE_target', 'rank_in_day', 'session_idx', 'conv_session_idx']
def remove_columns(df, columns_to_remove:list):
    return df.drop(columns=columns_to_remove)
    # we could remove it in place
df_unique = remove_columns(df_unique, Remove)

In [18]:
ProfileReport(df_unique)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████| 25/25 [00:00<00:00, 70232.82it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
def one_hot_encode(df_unique, cat_cols, dummy_na=False, drop_first=False):
    # Solo intenta encodar las columnas que sí existen en el dataframe
    cols_to_encode = [c for c in cat_cols if c in df_unique.columns]

    out = pd.get_dummies(
        df_unique.copy(),
        columns=cols_to_encode,
        prefix=cols_to_encode,
        prefix_sep="_",
        dummy_na=dummy_na,
        drop_first=drop_first
    )

    dummy_cols = {c: [col for col in out.columns if col.startswith(f"{c}_")] for c in cols_to_encode}
    return out, dummy_cols


cat_cols = ["BROWSER", "DEVICECATEGORY", "EXPERIENCENAME", "GEOREGION",
            "TRAFFICMEDIUM", "TRAFFICUSER", "TRAFFICSOURCE"]

encoded_df, dummy_cols = one_hot_encode(df_unique, cat_cols, dummy_na=True, drop_first=False)

all_dummy_cols = list(dict.fromkeys([c for cols in dummy_cols.values() for c in cols]))


In [20]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

def fill_fields(df, feature_cols, y, inv_map, objective="LOGINSTATUS", k=5):
    df = df.copy()

    # Keep only F/T as labeled data; treat None/NaN as missing

    train_mask = y.notna()
    test_mask  = y.isna()

    df_train = df.loc[train_mask, feature_cols]
    y_train  = y.loc[train_mask].astype(int)

    df_test  = df.loc[test_mask, feature_cols]

    # If nothing to predict or nothing to train on
    if df_test.empty or df_train.empty:
        return df

    # Choose a safe k
    k = min(k, len(df_train))  # k cannot exceed number of training rows
    knn = KNeighborsClassifier(n_neighbors=k)

    # Fit and predict
    knn.fit(df_train, y_train)
    y_pred = knn.predict(df_test)

    # Map back to F/T and fill
    df.loc[test_mask, objective] = pd.Series(y_pred, index=df_test.index).map(inv_map)

    return df


In [24]:
objective = 'VISITORCATEGORY'
y = encoded_df[objective].map({"P": 0, "C": 1})  # NaN stays NaN
inv_map = {0: "P", 1: "C"}
encoded_df = fill_fields(encoded_df, all_dummy_cols, y, inv_map, objective)

In [25]:
encoded_df

,VISITDATE,CLOUDSECTION,COMPANYSIZE,INDUSTRY,EVENTDURATIONSECONDS,FORMCOUNTRY,FORMCOUNTRYTIER,FORMCOMPANYSIZE,FORMEXPERIENCETYPE,FORMPRODUCTINTEREST,...,TRAFFICUSER_nan,TRAFFICSOURCE_bing,TRAFFICSOURCE_dashboard,TRAFFICSOURCE_duckduckgo,TRAFFICSOURCE_google,TRAFFICSOURCE_google search,TRAFFICSOURCE_https:/crm.com/campaign/smb-commerce,TRAFFICSOURCE_single,TRAFFICSOURCE_yahoo,TRAFFICSOURCE_nan
0,2024-12-10,Sales,None,software,353.0,US,1,None,None,None,...,False,False,False,False,True,False,False,False,False,False
1,2025-01-08,Platform,M,financial services,NaN,US,1,None,None,None,...,False,False,False,False,True,False,False,False,False,False
2,2025-01-23,Sales,None,None,5.0,US,1,None,None,None,...,False,True,False,False,False,False,False,False,False,False
3,2024-11-05,Small Business,None,None,7.0,US,1,10,Stacked,sales,...,False,False,False,False,True,False,False,False,False,False
4,2024-12-30,None,None,None,7.0,None,4,None,None,None,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4912,2024-10-22,None,None,None,106.0,None,4,None,None,None,...,False,False,True,False,False,False,False,False,False,False
4913,2025-01-05,None,None,None,1.0,None,4,None,None,None,...,False,False,False,False,True,False,False,False,False,False
4914,2024-11-06,Commerce,S,software,3.0,US,1,None,None,None,...,False,False,False,False,True,False,False,False,False,False
4915,2025-01-03,None,None,None,5.0,None,4,None,None,None,...,False,False,False,False,True,False,False,False,False,False


In [26]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# X: dataframe of features, y: target series
# Example:
X = encoded_df[all_dummy_cols]
y = encoded_df["CONVERSION"]

cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns
num_cols = X.columns.difference(cat_cols)


X2 = X.copy()

# 1) Asegura nombres únicos (por si aún hay duplicados)
X2 = X2.groupby(X2.columns, axis=1).max()

# 2) bool -> int (True/False -> 1/0)
bool_cols = X2.select_dtypes(include=["bool"]).columns
X2[bool_cols] = X2[bool_cols].astype("int8")



preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop"
)

model = RandomForestClassifier(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=1,
    class_weight="balanced",  # good default if classes are imbalanced
    n_jobs=-1,
    random_state=42
)

pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("rf", RandomForestClassifier(
        n_estimators=600,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced"
    ))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_validate(
    pipe, X2, y,
    cv=cv,
    scoring={"roc_auc":"roc_auc","f1":"f1","precision":"precision","recall":"recall","accuracy":"accuracy"},
    n_jobs=-1,
    return_train_score=False,
    error_score="raise"
)

/var/folders/5s/w5358_r11c15dhz1p9hjy2cr0000gn/T/ipykernel_10249/373052363.py:23: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  X2 = X2.groupby(X2.columns, axis=1).max()


In [27]:
scores

{'fit_time': array([3.21188879, 3.20940495, 3.24032903, 3.19783711, 3.15746093]),
 'score_time': array([0.57609606, 0.58772492, 0.57765913, 0.5778749 , 0.58546805]),
 'test_roc_auc': array([0.65212533, 0.63167254, 0.63901182, 0.64418592, 0.62673283]),
 'test_f1': array([0.23404255, 0.2170088 , 0.24855491, 0.24778761, 0.22043011]),
 'test_precision': array([0.16296296, 0.15744681, 0.17842324, 0.17948718, 0.15355805]),
 'test_recall': array([0.41509434, 0.3490566 , 0.40952381, 0.4       , 0.39047619]),
 'test_accuracy': array([0.70731707, 0.72865854, 0.73550356, 0.74059003, 0.70498474])}

In [28]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_fscore_support

proba = cross_val_predict(pipe, X2, y, cv=cv, method="predict_proba", n_jobs=-1)[:, 1]

ths = np.linspace(0.01, 0.99, 99)
best = None

for t in ths:
    pred = (proba >= t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y, pred, average="binary", zero_division=0)
    row = (t, p, r, f1)
    if best is None or row[3] > best[3]:
        best = row

print("best threshold, precision, recall, f1 =", best)


best threshold, precision, recall, f1 = (np.float64(0.35000000000000003), 0.16132596685082873, 0.5540796963946869, 0.24989302524604193)
